# Optimization projec: clustering

__Given that we don't have the proper datafiles to run the code, we will use synthetic ones to still do the work, and
as soon as we have the real files we will just substitute them.__

In [110]:
import numpy as np
from numpy import random
import pandas as pd
from sklearn.neighbors import DistanceMetric
#Simulate dataframe
X1 = np.random.uniform(low=0, high=25, size=(200,))
X2 = np.random.uniform(low=5, high=30, size=(200))
df = pd.DataFrame({'X1': X1, 'X2': X2}, columns=['X1', 'X2'])

In [112]:
df

,X1,X2
0,12.055290,13.237199
1,5.121922,27.063827
2,11.504386,25.360372
3,19.936433,11.807507
4,1.197959,6.096098
...,...,...
195,1.712261,19.343305
196,6.438160,7.891100
197,15.513548,26.715494
198,16.465455,25.216541


## Minimum Spanning Tree Clustering


In [115]:
threshold = []
step_size = 0
signal = 0
iteration = 1


AttributeError: 'DataFrame' object has no attribute 'row'